In [1]:
# Intended to address throughly the basics of statistics required for the course.
#Generalised Linear Models; the general linear model and the least-squares method, 
#logistic regression for binary responses, Poisson regression for count data. 
#More broadly, how to build a flexible linear predictor to capture relationships of interest. 

#Statistical Methods; commonly used probability distributions, 
#parameter estimation, sampling variability, hypothesis testing, 
#basic measures of bivariate relationships. 

library(tidyverse)
library(ggplot2)
library(aod)
library(reshape2)
library(lattice)
library(leaps)
library(vcd)
library(plotROC)
library(data.table)
require(boot)
library(broom)
require(sandwich)
require(msm)
library(ggExtra)
library(gridExtra)
list.files(path = "../input")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   0.8.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: grid


Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: boot


Attaching package: ‘boot’


The following object is masked from ‘package:lattice’:

    melanoma


Loading required package: sandwich

Loading r

character(0)